## Homework 7, solution

1. Following our discussion/confusion on the `r2_score`: think about the shortcomings of using the squared correlation coefficient as a quality of fit measure. Construct a few simple examples, that highlight, why `np.corrcoef(y_test,yHat)**2` would be a very poor choice.

2. Look up the sklearn help file for [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html#sklearn.preprocessing.PolynomialFeatures) and translate the statsmodel from below into the corresponding sklearn version, using `LinearRegression()` only.
3. Look up the `patsy` help file for [dmatrix](https://patsy.readthedocs.io/en/latest/formulas.html#redundancy-and-categorical-factors) and translate the statsmodel from below into the corresponding sklearn version, using `LinearRegression()` only.

4. Estimate the *10-fold* CV error on this model.



In [35]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
#import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

In [65]:
boston = pd.read_csv('../data/boston.csv')

#X = boston.drop('medv', axis=1)
y = boston['medv'].values
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [40]:
model = smf.ols('medv ~ lstat*rm + C(chas) + crim ', boston).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-29.8113,3.268,-9.121,0.000,-36.233,-23.390
C(chas)[T.1],3.2168,0.803,4.006,0.000,1.639,4.795
lstat,2.2458,0.201,11.170,0.000,1.851,2.641
rm,9.6980,0.489,19.843,0.000,8.738,10.658
lstat:rm,-0.4818,0.034,-14.346,0.000,-0.548,-0.416
crim,-0.1138,0.027,-4.278,0.000,-0.166,-0.062


### 1. r2_score

The correlation coefficient removes translations (intercepts) and scaling (slope), so it is not suitable as a measure of fit. 
We can start with two arrays with zero MSE. Any linear transformation of one of the two will still yield perfect correlation !

In [66]:
np.set_printoptions(precision=2)#driving me crazy

yObs = np.arange(10)
yHat = yObs
print("MSE:", mean_squared_error(yObs, yHat),  "cor:", np.round(np.corrcoef(yObs, yHat)[0,0],2))
yHat = 1 +2*yObs
print("MSE:", mean_squared_error(yObs, yHat),  "cor:", np.round(np.corrcoef(yObs, yHat)[0,0],2))


MSE: 0.0 cor: 1.0
MSE: 38.5 cor: 1.0


### 2. PolynomialFeatures

In [61]:
poly = PolynomialFeatures(interaction_only=True)
X_int = pd.DataFrame(poly.fit_transform(boston[["lstat", "rm"]]), 
                     columns = ["intercept","lstat", "rm", "lstat:rm"])

X_int[["chas", "crim"]] = boston[["chas", "crim"]]
X_int.head()

,intercept,lstat,rm,lstat:rm,chas,crim
0,1.0,4.98,6.575,32.74350,0,0.00632
1,1.0,9.14,6.421,58.68794,0,0.02731
2,1.0,4.03,7.185,28.95555,0,0.02729
3,1.0,2.94,6.998,20.57412,0,0.03237
4,1.0,5.33,7.147,38.09351,0,0.06905


In [68]:
regr = LinearRegression().fit(X_int, y)
pd.DataFrame(regr.coef_, index=X_int.columns)

,0
intercept,0.000000
lstat,2.245841
rm,9.697983
lstat:rm,-0.481807
chas,3.216785
crim,-0.113761


### 3. dmatrix from patsy

In [69]:
X= patsy.dmatrix('lstat*rm + C(chas) + crim ', boston)

In [70]:
regr = LinearRegression().fit(X, y)
pd.DataFrame(regr.coef_, index=X.design_info.term_names)

,0
Intercept,0.000000
C(chas),3.216785
lstat,2.245841
rm,9.697983
lstat:rm,-0.481807
crim,-0.113761


### 4. Cross Validation

In [33]:

reg_all = LinearRegression()

cv_results = cross_val_score(reg_all, X, y, cv=10, scoring="neg_mean_squared_error")
print(np.round(cv_results,2))

[  -7.75   -6.02   -5.31  -27.95  -13.05  -22.36  -13.7  -120.99  -29.85
  -13.87]


In [32]:
import sklearn

sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']